# LAB 3: How to setup a project from Scratch

# Step 1: Clone your project from Github

In [1]:
# !git clone https://github.com/lambdavi/mldl_project_skeleton.git
!git clone https://github.com/SarvnazRmn/mldl_project_skeleton.git


Cloning into 'mldl_project_skeleton'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 22 (delta 5), reused 5 (delta 1), pack-reused 11 (from 2)
Receiving objects: 100% (22/22), 7.09 KiB | 7.09 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!ls

mldl_project_skeleton  sample_data


In [3]:
%cd mldl_project_skeleton

/content/mldl_project_skeleton


# Step 2: Packages Installation


In [4]:
!pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [4]:
# !python utils/download_dataset.py

!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip -d tiny-imagenet

import os
import shutil

# Note: This entire code block is necessary to make the validation data readable by ImageFolder
with open('tiny-imagenet/tiny-imagenet-200/val/val_annotations.txt') as f:
    for line in f:
        fn, cls, *_ = line.split('\t')
        os.makedirs(f'tiny-imagenet/tiny-imagenet-200/val/{cls}', exist_ok=True)
        shutil.copyfile(f'tiny-imagenet/tiny-imagenet-200/val/images/{fn}', f'tiny-imagenet/tiny-imagenet-200/val/{cls}/{fn}')

# 4. Remove the redundant folder after reorganization
shutil.rmtree('tiny-imagenet/tiny-imagenet-200/val/images')

print("Data setup complete and validation structure fixed.")

--2025-11-02 20:08:57--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-11-02 20:08:57--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  20.3MB/s    in 13s     

2025-11-02 20:09:11 (18.8 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

Data setup complete and validation structure fixed.


In [5]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

# t.compsoe is applied to all images
transform = T.Compose([
    T.Resize((224, 224)),  # Tiny ImageNet images are $64*  64$ pixels,this architecture recieve 24 as standard input
    T.ToTensor(),   #Converts the image data into a PyTorch Tensor.
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #adjusts the image's colors/intensities for scaled GD.
])

# root/{classX}/x001.jpg
#define dataset
tiny_imagenet_dataset_train = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/train', transform=transform)
tiny_imagenet_dataset_val = ImageFolder(root='tiny-imagenet/tiny-imagenet-200/val', transform=transform)

import torch
#define dataloader
train_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_train, batch_size=32, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(tiny_imagenet_dataset_val, batch_size=32, shuffle=False)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Step 4: Train your model and visualize training

In [6]:
%env WANDB_API_KEY=<your_api_key>

env: WANDB_API_KEY=<your_api_key>


In [7]:
!python train.py

Traceback (most recent call last):
  File "/content/mldl_project_skeleton/train.py", line 1, in <module>
    from models.customnet import CustomNet
ModuleNotFoundError: No module named 'models.customnet'


# Step 5: Evaluate your model

1.   List item
2.   List item



In [8]:
!python eval.py

Traceback (most recent call last):
  File "/content/mldl_project_skeleton/eval.py", line 1, in <module>
    from models.custom import CustomNet
ModuleNotFoundError: No module named 'models.custom'


In [ ]:
#MODEL INSTANCIATION

import torch

from torch import nn
from models.customnet import CustomNet
from train import train
from eval import validate

model = CustomNet().cuda()   #TRANSFER ALL PARAM AND WEIGHTS TO GPU
criterion = nn.CrossEntropyLoss()   #CRITERIAON IS LOSS
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)   # optimzer is set to SGD

#MOMENTUM = A parameter that helps the optimizer move faster and more steadily towards the optimum,




best_acc = 0   #track the highest validation accuracy

# Run the training process for {num_epochs} epochs
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(epoch, model, train_loader, criterion, optimizer)

    # At the end of each training iteration, perform a validation step
    val_accuracy = validate(model, val_loader, criterion)

    # Best validation accuracy
    best_acc = max(best_acc, val_accuracy)


print(f'Best validation accuracy: {best_acc:.2f}%')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train Epoch: 1 Loss: 4.504814 Acc: 9.70%
Validation Loss: 4.083897 Acc: 15.65%
Train Epoch: 2 Loss: 3.641662 Acc: 22.33%
Validation Loss: 3.727363 Acc: 20.58%
Train Epoch: 3 Loss: 2.644468 Acc: 39.75%
Validation Loss: 3.893978 Acc: 20.52%
Train Epoch: 4 Loss: 0.995896 Acc: 75.39%
Validation Loss: 5.154020 Acc: 17.01%
Train Epoch: 5 Loss: 0.181941 Acc: 95.51%
Validation Loss: 6.688257 Acc: 16.74%
Train Epoch: 6 Loss: 0.055051 Acc: 98.87%
Validation Loss: 7.007783 Acc: 17.82%
Train Epoch: 7 Loss: 0.022914 Acc: 99.69%
Validation Loss: 7.386923 Acc: 18.31%
Train Epoch: 8 Loss: 0.016036 Acc: 99.88%
Validation Loss: 7.310565 Acc: 18.45%
Train Epoch: 9 Loss: 0.012503 Acc: 99.95%
Validation Loss: 7.440836 Acc: 19.12%
